# Généralisation de la méthode de la puissance

## Calcul de la plus petite valeur propre

Une première généralisation de la méthode de la puissance consiste à l&#8217;appliquer à l&#8217;inverse de la matrice A (à condition bien sûr que A soit inversible!). Comme les valeurs propres de $\mathrm{A}^{-1}$ sont les inverses de celles de A, la méthode de la puissance nous permet alors d&#8217;approcher la valeur propre de A de plus petit module. C&#8217;est la méthode de la puissance inverse :
étant donné un vecteur $\mathbf{x}^{(0)}$, on pose $\mathbf{y}^{(0)}=\mathbf{x}^{(0)} /\left\|\mathbf{x}^{(0)}\right\|$ et on calcule

$$
\left|\begin{array}{l}
\text { pour } k=1,2, \ldots \\
\mathbf{x}^{(k)}=\mathrm{A}^{-1} \mathbf{y}^{(k-1)}, \mathbf{y}^{(k)}=\frac{\mathbf{x}^{(k)}}{\left\|\mathbf{x}^{(k)}\right\|}, \mu^{(k)}=\left(\mathbf{y}^{(k)}\right)^H \mathrm{~A}^{-1} \mathbf{y}^{(k)}
\end{array}\right|
$$
Si les vecteurs propres de A sont linéairement indépendants, et s&#8217;il n&#8217;y a qu&#8217;une valeur propre $\lambda_n$ de module minimal, alors

$$
\lim _{k \rightarrow \infty} \mu^{(k)}=1 / \lambda_n
$$
i.e. $\left(\mu^{(k)}\right)^{-1}$ tend vers $\lambda_n$ pour $k \rightarrow \infty$.
A chaque étape $k$, on doit résoudre un système linéaire de la forme $\mathrm{A} \mathbf{x}^{(k)}=\mathbf{y}^{(k-1)}$.
*Important:* Il est donc commode d&#8217;effectuer une factorisation LU de A (ou une factorisation de Cholesky si $A$ est symétrique définie positive) une fois pour toute, afin de n&#8217;avoir à résoudre que deux systèmes triangulaires à chaque itération.
*Note:* Rappelons que la commande lu  peut également effectuer la décomposition LU pour des matrices complexes.

## Calcul d&#8217;une valeur propre proche d&#8217;une valeur donnée

Une autre généralisation de la méthode de la puissance permet de calculer une approximation de la valeur propre (inconnue) la plus proche d&#8217;un $\mu$ donné (réel ou complexe).
Notons $\lambda_\mu$ une telle valeur propre et définissons la matrice translatée $\mathrm{A}_\mu=\mathrm{A}-\mu \mathrm{I}$, dont les valeurs propres sont $\lambda\left(A_\mu\right)=\lambda(A)-\mu$.
Pour approcher $\lambda_\mu$, on peut d&#8217;abord estimer $\lambda_{\text {min }}\left(\mathrm{A}_\mu\right)$, valeur propre de plus petite norme de $\mathrm{A}_\mu$, en appliquant la méthode de la puissance inverse à $\mathrm{A}_\mu$, puis calculer $\lambda_\mu=\lambda_{\text {min }}\left(\mathrm{A}_\mu\right)+\mu$. Cette technique est connue sous le nom de **méthode de la puissance avec décalage ou avec translation** (shift en anglais), et le nombre $\mu$ est appelé décalage (ou shift).

## Implémentation

On implémente la méthode de la puissance inverse avec décalage.
Le paramètre d&#8217;entrée $\mu$ est le décalage, les autres sont identiques à ceux de `eigpower`.
La méthode de la puissance inverse (sans décalage) est simplement obtenue en prenant $\mu=0$.
Les paramètres de sortie sont la valeur propre approchée $\lambda_\mu$ de A, un vecteur propre associé $\mathbf{x}$ et le nombre d&#8217;itérations effectuées.
```python#invshift
Unresolved directive in 3-puissance-generalisee.adoc - include::example$tan/eig/invshift.py[]
```## Exemples

*Exemple: famille de matrice $A(\alpha)$*\
Appliquons la méthode de la puissance inverse pour calculer la valeur propre de plus petit module de la matrice $A(30)$ définie dans [l&#8217;exemple du chapitre précédent](chap9/2-puissance.ipynb#ex:1). [le programme](#invshift) appelé de la manière suivante :


In [0]:
import numpy as np
from tan.eig.invshift import invshift

# Matrice A pour alpha = 30
alpha = 30
A = np.array([[alpha, 2, 3, 13],
              [5, 11, 10, 8],
              [9, 7, 6, 12],
              [4, 14, 15, 1]])

l, x, iter=invshift(A,tol=1.e-10,x0=np.ones(4))
print(f"converge en {iter} itérations vers la valeur {l:.4f} .")


*Exemple: famille de matrice $A(\alpha)$*\
Toujours pour la matrice A(30), on cherche la valeur propre la plus proche de 17.
On utilise pour cela `invshift` avec $\mathrm{mu}=17$, tol $=10^{-10}$ et $x 0=[1 ; 1 ; 1 ; 1$].
Après 8 itérations, le programme retourne la valeur lambda $=17.82079703055703$.

Une connaissance moins précise du décalage aurait entraîné plus d&#8217;itérations.
Par exemple, si on pose $m u=13$ on obtient la valeur 17.82079703064106 après 19 itérations.

*Note:* On peut modifier la valeur du décalage au cours des itérations, en posant $\mu=\lambda^{(k)}$. Ceci accélère la convergence mais augmente significativement le coût de chaque itération puisque la matrice $\mathrm{A}_\mu$ doit être refactorisée à chaque modification de $\mu$.
## Comment déterminer le décalage ?

Pour que la méthode de la puissance avec décalage soit efficace, il faut localiser (plus ou moins précisément) les valeurs propres de $A$ dans le plan complexe. Commençons par quelques définitions.
Soit A une matrice carrée d&#8217;ordre $n$. Les disques de Gershgorin $C_i^{(r)}$ et $C_i^{(c)}$ associés à la $i$-ème ligne et à la $i$-ème colonne sont respectivement définis par

$$
\begin{aligned}
& C_i^{(r)}=\left\{z \in \mathbb{C}:\left|z-a_{i i}\right| \leq \sum_{j=1, j \neq i}^n\left|a_{i j}\right|\right\}, \\
& C_i^{(c)}=\left\{z \in \mathbb{C}:\left|z-a_{i i}\right| \leq \sum_{j=1, j \neq i}^n\left|a_{j i}\right|\right\} .
\end{aligned}
$$
$C_i^{(r)}$ est le disque de la $i$-ème ligne et $C_i^{(c)}$ celui de la $i$-ème colonne.
On peut visualiser les cercles de Gershgorin associés à une matrice A en utilisant la fonction `gershgorin_circles` de la librairie `tan.eig.gershgorin`.
```python#gershgorin
Unresolved directive in 3-puissance-generalisee.adoc - include::example$tan/eig/gershgorin.py[]
```*Exemple: Tracé des disques de Gershgorin*\
On a tracé les disques de Gershgorin associés à la matrice


$$
A=\left[\begin{array}{cccc}
30 & 1 & 2 & 3 \\
4 & 15 & -4 & -2 \\
-1 & 0 & 3 & 5 \\
-3 & 5 & 0 & -1
\end{array}\right]
$$

Les centres des disques sont repérés par une croix.


In [0]:
from tan.eig.gershgorin import gershgorin_circles
import numpy as np
# Matrice A exemple
A = np.array([
    [30, 1, 2, 3],
    [4, 15, -4, -2],
    [-1, 0, 3, 5],
    [-3, 5, 0, -1]
], dtype=np.complex_)

fig = gershgorin_circles(A)
fig.show()


Les disques de Gershgorin peuvent servir à localiser les valeurs propres d&#8217;une matrice, comme le montre la proposition suivante
*Proposition: Localiser les valeurs propres avec les disques de Gershgorin*\
Toutes les valeurs propres d&#8217;une matrice $A \in$ $\mathbb{C}^{n \times n}$ appartiennent à la région du plan complexe définie par l&#8217;intersection des deux régions constituées respectivement de la réunion des disques des lignes et des disques des colonnes.
Si de plus $m$ disques des lignes (ou des colonnes), $1 \leq m \leq n$, sont disjoints de la réunion des $n-m$ autres disques, alors leur réunion contient exactement $m$ valeurs propres.
Rien n&#8217;assure qu&#8217;un disque contienne des valeurs propres, à moins qu&#8217;il ne soit isolé des autres.
L&#8217;estimation fournie par les disques de Ghersghorin est en général assez grossière. On peut cependant utiliser le résultat ci-dessus pour avoir une première estimation du décalage, comme le montre l&#8217;exemple suivant.
*Note:* Remarquer qu&#8217;on peut déduire de la Proposition 6.1 que toutes les valeurs propres d&#8217;une matrice à diagonale strictement dominante sont non nulles.
*Exemple: famille de matrice $A(\alpha)$*\
On déduit de l&#8217;analyse des disques des lignes de la matrice $A(30)$ de l&#8217;Exemple précédent, que les parties réelles des valeurs propres de A sont comprises entre -32 et 48.
On peut donc utiliser `invshift` pour calculer la valeur propre de module maximal en choisissant un décalage $\mu$ égal à 48.
La méthode converge alors en 15 itérations, tandis que 22 itérations sont nécessaires pour la méthode de la puissance avec la même donnée initiale $x 0=[1 ; 1 ; 1 ; 1]$ et la même tolérance $tol=1.e-10$.

## Résumé


1. La ***méthode de la puissance*** est un algorithme itératif qui permet le calcul de la valeur propre de plus grand module d&#8217;une matrice donnée;
1. la ***méthode de la puissance inverse*** permet le calcul de la valeur propre de plus petit module; pour l&#8217;implémenter efficacement, il est recommandé de factoriser la matrice avant de démarrer les itérations ;
1. la ***méthode de la puissance avec décalage*** permet le calcul de la valeur propre la plus proche d&#8217;une valeur donnée; pour être efficace, elle nécessite une connaissance a priori de la localisation des valeurs propres de la matrice. Cette localisation peut se faire à l&#8217;aide des disques de Gershgorin.

